In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU model: {torch.cuda.get_device_name(0)}")
    print(f"GPU device count: {torch.cuda.device_count()}")
else:
    print("No GPU available.")
  

GPU model: Tesla T4
GPU device count: 1


# Data Pre-processing

First we will begin by pre-processing the data. In this case the sample data comes with the following features/columns: 

|   **Column**   |   **Desciption**   |
|:-:	         |:--	              |
|   STUD	|   Study Number 	|
|   DSFQ	|   Dosing Frequency	|
|   PTNM	|   Patient Number	|
|   CYCL	|    Dosing Cycles	|
|   AMT	|   Dosing Amounts	|
|   TIME	|   Time in Hours Since the Experiment Began for one Individual	|
|   TFDS	|   Time in Hours Since the Last Dosing|
|   DV/PK_timeCourse	|   The Observations of PK  	|

Let us first import the data into the notebook and observe these features of the dataset

In [2]:
from google.colab import files
 
 
uploaded = files.upload()

Saving sim_data.txt to sim_data.txt


In [4]:
# Importing required libraries for data pre-processing

import pandas as pd
import numpy as np
import io


#Reading the csv file with the data

data_complete = pd.read_csv(io.BytesIO(uploaded['sim_data.txt']))
data_complete


# data_complete = pd.read_csv("/Users/rishabhgoel/Desktop/NeuralODE_Paper_Supplementary_Code/ExampleData/sim_data.csv", na_values='.')

# data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,DV
0,1000.0,1.0,3.0,1.0,296.6,0.0,0.0,20.382000
1,1000.0,1.0,3.0,1.0,0.0,24.0,24.0,73.148000
2,1000.0,1.0,3.0,1.0,0.0,216.0,216.0,19.764000
3,1000.0,1.0,3.0,1.0,0.0,504.0,504.0,3.219900
4,1000.0,1.0,3.0,2.0,288.0,504.0,0.0,3.219900
...,...,...,...,...,...,...,...,...
5371,3000.0,200.0,3.0,15.0,0.0,7416.0,360.0,6.091200
5372,3000.0,200.0,3.0,15.0,0.0,7584.0,7584.0,2.043100
5373,3000.0,200.0,3.0,17.0,259.2,8064.0,0.0,0.090115
5374,3000.0,200.0,3.0,17.0,0.0,8088.0,24.0,53.990000


There are 5376 observations and 8 features that we are looking at. Now let us begin the pre-processing the data. 

Our sample data only has the relevant columns for the model. However, in the data we receive from patients there will be many more features so it is important for us to select the correct features. Thus, let us begin by creating a place holder for the features that are important to the model.

In [5]:
#variable for colummns that we will eventually select from the raw dataset we receive

select_cols = ["STUD", "DSFQ", "PTNM", "CYCL", "AMT", "TIME", "TFDS", "DV"]

#Selecting the relevant columns from the dataframe

data_complete = data_complete[select_cols]

We then start filtering data based on multiple parameters:
    
1. Dosing Cycle < 100

In [6]:
# filtering for only the rows in the dataframe with a dosing cycle less than 100

data_complete = data_complete[data_complete.CYCL < 100]
data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,DV
0,1000.0,3.0,1.0,1.0,296.6,0.0,0.0,20.382000
1,1000.0,3.0,1.0,1.0,0.0,24.0,24.0,73.148000
2,1000.0,3.0,1.0,1.0,0.0,216.0,216.0,19.764000
3,1000.0,3.0,1.0,1.0,0.0,504.0,504.0,3.219900
4,1000.0,3.0,1.0,2.0,288.0,504.0,0.0,3.219900
...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,200.0,15.0,0.0,7416.0,360.0,6.091200
5372,3000.0,3.0,200.0,15.0,0.0,7584.0,7584.0,2.043100
5373,3000.0,3.0,200.0,17.0,259.2,8064.0,0.0,0.090115
5374,3000.0,3.0,200.0,17.0,0.0,8088.0,24.0,53.990000


We then convert the "PTNM" column to an integer type using the astype() method, and then using the map() method to apply a format string to each value in the column. The format string "{:05d}" specifies that each value should be formatted as a zero-padded integer with a width of 5 digits.

For example, if the "PTNM" column originally contained the values [1, 10, 100, 1000], after this line of code is executed, the "PTNM" column would contain the values ['00001', '00010', '00100', '01000'].

This line of code is useful for standardizing the format of the values in the "PTNM" column, which can make it easier to perform operations on the column and to compare values within the column. It can also be useful for preparing the data for downstream model applications that require the data to be in a particular format.

In [7]:
# formatting Patient Number for Unique Identifier
data_complete["PTNM"] = data_complete["PTNM"].astype("int").map("{:05d}".format)
data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,DV
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.382000
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.148000
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.764000
3,1000.0,3.0,00001,1.0,0.0,504.0,504.0,3.219900
4,1000.0,3.0,00001,2.0,288.0,504.0,0.0,3.219900
...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,00200,15.0,0.0,7416.0,360.0,6.091200
5372,3000.0,3.0,00200,15.0,0.0,7584.0,7584.0,2.043100
5373,3000.0,3.0,00200,17.0,259.2,8064.0,0.0,0.090115
5374,3000.0,3.0,00200,17.0,0.0,8088.0,24.0,53.990000


Now we create a new column called "ID" in the pandas DataFrame data_complete. The "ID" column is being created by concatenating two existing columns in ```data_complete```: "STUD" and "PTNM".

First, the "STUD" column is being converted to an integer data type using the ```astype()``` method with the argument "int". Then, the "STUD" column is being converted to a string data type using the ```astype()``` method with the argument "str". This ensures that the values in the "STUD" column are in string format.

Next, the values in the "STUD" and "PTNM" columns are being concatenated using the + operator, which joins the two strings together. This creates a new string for each row in the DataFrame, which is assigned to the "ID" column.

For example, if the "STUD" column contained the values ['100', '101', '102', '103'] and the "PTNM" column contained the values ['00001', '00010', '00100', '01000'], then after this line of code is executed, the "ID" column would contain the values ['10000001', '10100010', '10200100', '10301000'].

This line of code is useful for creating a unique identifier for each row in the DataFrame based on the values in the "STUD" and "PTNM" columns. This can be useful for identifying and tracking individual records, as well as for linking data across multiple datasets.

In [8]:
# Creating a unique identifier column, 'ID', and formatting it based on Patient Number and Study Number
data_complete["ID"] = data_complete["STUD"].astype("int").astype("str") + data_complete["PTNM"]
data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,DV,ID
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.382000,100000001
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.148000,100000001
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.764000,100000001
3,1000.0,3.0,00001,1.0,0.0,504.0,504.0,3.219900,100000001
4,1000.0,3.0,00001,2.0,288.0,504.0,0.0,3.219900,100000001
...,...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,00200,15.0,0.0,7416.0,360.0,6.091200,300000200
5372,3000.0,3.0,00200,15.0,0.0,7584.0,7584.0,2.043100,300000200
5373,3000.0,3.0,00200,17.0,259.2,8064.0,0.0,0.090115,300000200
5374,3000.0,3.0,00200,17.0,0.0,8088.0,24.0,53.990000,300000200


In [9]:
# Creating a dataframe with the maximum time for each patient in each study (Essentially presenting the time of the last observed dose)
time_summary = data_complete[["ID", "TIME"]].groupby("ID").max().reset_index()

# Creating a new dataframe with only the IDs for patients with a time of the last observed dose greater than 0 (eliminating errors/outliers)
selected_ptnms = time_summary[time_summary.TIME > 0].ID

# Only selecting IDs with the last observed dose greater than 0 in our main table called data_complete
data_complete = data_complete[data_complete.ID.isin(selected_ptnms)]
data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,DV,ID
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.382000,100000001
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.148000,100000001
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.764000,100000001
3,1000.0,3.0,00001,1.0,0.0,504.0,504.0,3.219900,100000001
4,1000.0,3.0,00001,2.0,288.0,504.0,0.0,3.219900,100000001
...,...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,00200,15.0,0.0,7416.0,360.0,6.091200,300000200
5372,3000.0,3.0,00200,15.0,0.0,7584.0,7584.0,2.043100,300000200
5373,3000.0,3.0,00200,17.0,259.2,8064.0,0.0,0.090115,300000200
5374,3000.0,3.0,00200,17.0,0.0,8088.0,24.0,53.990000,300000200


In [10]:
# filling in missing values in the "AMT" column with Dosing Amounts equal to 0
data_complete["AMT"] = data_complete["AMT"].fillna(0)


#Renaming the column DV to PK_timeCourse so that the name is more self-explanatory
data_complete = data_complete.rename(columns={"DV": "PK_timeCourse"})


# Duplicating PK_timeCourse column to make changes to it separately
data_complete["PK_round1"] = data_complete["PK_timeCourse"]

# Changing the PK_round1 such that if the dosing frequency is once weekly and the time of the last 
# observed dose is greater than 168hrs the PK for round 1 is zero. Similarly, if the dosing frequency 
# is once weekly and the time of the last observed dose is greater than 504hrs the PK for round 1 is zero
data_complete.loc[(data_complete.DSFQ == 1) & (data_complete.TIME >= 168), "PK_round1"] = 0
data_complete.loc[(data_complete.DSFQ == 3) & (data_complete.TIME >= 504), "PK_round1"] = 0

# filling in missing values in the "PK_round1" column with 0
data_complete["PK_round1"] = data_complete["PK_round1"].fillna(0)

# filling in missing values in the "PK_timeCourse" column with -1
data_complete["PK_timeCourse"] = data_complete["PK_timeCourse"].fillna(-1)
data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID,PK_round1
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.382000,100000001,20.382
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.148000,100000001,73.148
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.764000,100000001,19.764
3,1000.0,3.0,00001,1.0,0.0,504.0,504.0,3.219900,100000001,0.000
4,1000.0,3.0,00001,2.0,288.0,504.0,0.0,3.219900,100000001,0.000
...,...,...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,00200,15.0,0.0,7416.0,360.0,6.091200,300000200,0.000
5372,3000.0,3.0,00200,15.0,0.0,7584.0,7584.0,2.043100,300000200,0.000
5373,3000.0,3.0,00200,17.0,259.2,8064.0,0.0,0.090115,300000200,0.000
5374,3000.0,3.0,00200,17.0,0.0,8088.0,24.0,53.990000,300000200,0.000


In [11]:
# Removing rows where the "AMT" column is 0 and the "TIME" column is also 0.
data_complete = data_complete[~((data_complete.AMT == 0) & (data_complete.TIME == 0))]

# Keeping the last row for all patients with duplicate values for time in hours 
data_complete.loc[data_complete[["PTNM", "TIME"]].duplicated(keep="last"), "AMT"] = \
    data_complete.loc[data_complete[["PTNM", "TIME"]].duplicated(keep="first"), "AMT"].values

# Keeping the first row for all observations with duplicate values for all features apart from patient and time in hours 
data_complete = data_complete[~data_complete[["PTNM", "TIME"]].duplicated(keep="first")]

data_complete

,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID,PK_round1
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.382000,100000001,20.382
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.148000,100000001,73.148
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.764000,100000001,19.764
3,1000.0,3.0,00001,1.0,288.0,504.0,504.0,3.219900,100000001,0.000
5,1000.0,3.0,00001,2.0,0.0,696.0,192.0,23.210000,100000001,0.000
...,...,...,...,...,...,...,...,...,...,...
5371,3000.0,3.0,00200,15.0,0.0,7416.0,360.0,6.091200,300000200,0.000
5372,3000.0,3.0,00200,15.0,0.0,7584.0,7584.0,2.043100,300000200,0.000
5373,3000.0,3.0,00200,17.0,259.2,8064.0,0.0,0.090115,300000200,0.000
5374,3000.0,3.0,00200,17.0,0.0,8088.0,24.0,53.990000,300000200,0.000


# Splitting Data

Now, we want to be able to split our data into training, validation, and test groups. This is a method of ensuring that we can hypertune our model without any biases and have the most robust version of our model in the final output.

We begin by defining a function called ```data_split``` that we will use eventually to split our data in a customized way. Arguments for the function include a dataframe (```df```), a column (```on_col```), a list of columns to be saved in the output dataframes (```save_cols```), the seed of the split (```seed```), and the proportion of the data that will go into the test set (```test_size```).

In [12]:
#Importing the necessary library for splitting the data into training and test sets
from sklearn.model_selection import train_test_split

#Defining the function called data_split which takes in a dataframe, a column name (likely with patient identifiers), 
# and a default test size of 20%. 
def data_split(df, on_col, save_cols=None, seed=2020, test_size=0.2):
    
    #Setting the default save_cols to all columns of the dataset
    if not save_cols:
        save_cols = df.columns.values

    #Setting a variable called target which contains all the unique values of a certain column. 
    #This variable will be helpful to split the unique patients into train and test later in the code.
    target = df[on_col].unique()
    
    #Setting the train variable to contain the random unique patients numbers of 80% of the patients 
    #while the remaining 20% is the allocated to the test variable
    train, test = train_test_split(target, random_state=seed, test_size=test_size, shuffle=True)
    
    #Creating the train and test dataframes (train_df and test_df). The training dataframe (train_df) includes all rows of 
    #patients in the train variable (train). The test dataframe (test_df) includes all rows of remaining patients (test).
    train_df = df[df[on_col].isin(train)]
    test_df = df[df[on_col].isin(test)]

    #Returning  
    return train_df[save_cols], test_df[save_cols]

We then build the code for the ```main``` that takes in our input data uses the helper function that we created above ```data_split()``` to split our data into training, validation, and test sets.

Next, the main adds a specific portion of the test set into the training and validation sets that we created above. 

Then, the main performs an augmentation that splits each patient into three groups. The main purpose of this augmentation in my opinion is to eliminate the problems associated with translating the results from one dosing regimen to the next.


In [13]:
def datasplitter(data, fold, model):
  train, test = data_split(data, "PTNM", seed=1329+fold, test_size=0.2)
  train, validate = data_split(train, "PTNM", seed=1329+fold+model, test_size=0.2)
  train = pd.concat([train, test[test.TIME < 168]], ignore_index=True)
  validate = pd.concat([validate, test[test.TIME < 168]], ignore_index=True)

  # James' augmentation
  # Deep learning is prone to overfitting, and they applied augmentation to prevent overfitting. We applied 
  # timewise truncation to increase the number of training examples. For each training example, in addition to the 
  # original example, we also truncated the examples at 1008 hr, 1512 hr, and 2016 hr and generated and added a 
  # set of new examples to the training examples.

  augment_data = pd.DataFrame(columns=train.columns)

  for ptnm in train.PTNM.unique():
      df = train[(train.PTNM == ptnm) & (train.TIME <= 2 * 21 * 24) & (train.TIME >= 0)]
      df["PTNM"] = df["PTNM"] + str(0.1)
      augment_data = pd.concat([augment_data, df], ignore_index=True)

      df = train[(train.PTNM == ptnm) & (train.TIME <= 3 * 21 * 24) & (train.TIME >= 0)]
      df["PTNM"] = df["PTNM"] + str(0.2)
      augment_data = pd.concat([augment_data, df], ignore_index=True)

      df = train[(train.PTNM == ptnm) & (train.TIME <= 4 * 21 * 24) & (train.TIME >= 0)]
      df["PTNM"] = df["PTNM"] + str(0.3)
      augment_data = pd.concat([augment_data, df], ignore_index=True)

  train = pd.concat([train, augment_data], ignore_index=True).reset_index(drop=True)
  

  train.to_csv("train.csv", index=False) 
  validate.to_csv("validate.csv", index=False)
  test.to_csv("test.csv", index=False)
  return train, validate, test

In [ ]:
# for fold in np.arange(1,6):
#   for model in np.arange(1,6):
#      train, validate, test = datasplitter(data_complete, fold, model)
#     #  runtest()
#      print(fold, model)

In [14]:
import pandas as pd
import numpy as np

def generate_interp(ptnm):
    test = pd.read_csv("test.csv")
    subset = test[test.PTNM == ptnm]
    interp_times = pd.DataFrame({"TIME": np.linspace(subset.loc[subset.TIME >= 168, "TIME"].values[:-1], \
                             subset.loc[subset.TIME >= 168, "TIME"].values[1:], 6, endpoint=False).flatten("F")})
    subset = pd.merge(subset, interp_times, how="outer", on="TIME").sort_values(by="TIME").reset_index(drop=True)
    subset["AMT"] = subset["AMT"].fillna(0)
    subset["PK_round1"] = subset["PK_round1"].fillna(0)
    subset["PTNM"] = subset["PTNM"].fillna(method="ffill")
    subset["DSFQ"] = subset["DSFQ"].fillna(1)
    subset["CYCL"] = subset["CYCL"].fillna(method="ffill")
    subset["PK_timeCourse"] = subset["PK_timeCourse"].fillna(-10) 
    
    start_time = 0
    for idx, feature in subset.iterrows():

        if feature["AMT"] > 0:
            start_time = feature["TIME"]
            continue

        subset.loc[idx, "TFDS"] = subset.loc[idx, "TIME"] - start_time

    return subset.drop(columns=["STUD", "ID"])

#ENDDDDDDD


# BRINGING IN THE ENTIRE UTILS.PY FILE

In [15]:
!pip install torchdiffeq
import os
import logging
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from torchdiffeq import odeint_adjoint as odeint


def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)


def get_device(tensor):
    device = torch.device("cuda")
    if tensor.is_cuda:
        device = tensor.get_device()
    return device


def load_model(ckpt_path, encoder=None, ode_func=None, classifier=None, device="cuda"):
    if not os.path.exists(ckpt_path):
        raise Exception("Checkpoint " + ckpt_path + " does not exist.")

    checkpt = torch.load(ckpt_path)
    if encoder is not None:
        encoder_state = checkpt["encoder"]
        encoder.load_state_dict(encoder_state)
        encoder.to(device)

    if ode_func is not None:
        ode_state = checkpt["ode"]
        ode_func.load_state_dict(ode_state)
        ode_func.to(device)

    if classifier is not None:
        classifier_state = checkpt["classifier"]
        classifier.load_state_dict(classifier_state)
        classifier.to(device)


def get_logger(logpath, filepath, package_files=[], displaying=True, saving=True, debug=False):
    logger = logging.getLogger()
    if debug:
        level = logging.DEBUG
    else:
        level = logging.INFO
    logger.setLevel(level)
    if saving:
        info_file_handler = logging.FileHandler(logpath, mode="a")
        info_file_handler.setLevel(level)
        logger.addHandler(info_file_handler)
    if displaying:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(level)
        logger.addHandler(console_handler)
    logger.info(filepath)
    """
    with open(filepath, "r") as f:
        logger.info(f.read())

    for f in package_files:
        logger.info(f)
        with open(f, "r") as package_f:
            logger.info(package_f.read())
    """

    return logger


def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()


def init_network_weights(net, std = 0.1):
    for m in net.modules():
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, mean=0, std=std)
            nn.init.constant_(m.bias, val=0)


def reverse(tensor):
    idx = [i for i in range(tensor.size(0)-1, -1, -1)]
    return tensor[idx]


def sample_standard_gaussian(mu, sigma):
    device = get_device(mu)
    d = torch.distributions.normal.Normal(
            torch.Tensor([0.]).to(device),
            torch.Tensor([1.]).to(device))
    r = d.sample(mu.size()).squeeze(-1)
    return r * sigma.float() + mu.float()


def compute_loss_on_train(criterion, labels, preds):
    preds = preds.permute(1, 0, 2)

    idx_not_nan = ~(torch.isnan(labels).to(device) | (labels == -1))
    # print(idx_not_nan)
    preds = preds[idx_not_nan]
    labels = labels[idx_not_nan]

    return torch.sqrt(criterion(preds, labels)).to(device)


def compute_loss_on_test(encoder, ode_func, classifier, dataloader, n_batches, device, phase):
    ptnms = []
    Times = torch.Tensor([]).to(device=device)
    predictions = torch.Tensor([]).to(device=device)
    ground_truth = torch.Tensor([]).to(device=device)
    latent_dim = 6

    for itr in range(n_batches):
        ptnm, times, features, labels, cmax_time = dataloader.__next__()
        dosing = torch.zeros([features.size(0), features.size(1), latent_dim]).to(device)
        dosing[:, :, 0] = features[:, :, -2]
        dosing = dosing.permute(1, 0, 2).to(device)

        encoder_out = encoder(features)
        qz0_mean, qz0_var = encoder_out[:, :latent_dim], encoder_out[:, latent_dim:]
        z0 = sample_standard_gaussian(qz0_mean, qz0_var)

        solves = z0.unsqueeze(0).clone()
        try:
            for idx, (time0, time1) in enumerate(zip(times[:-1], times[1:])):
                z0 += dosing[idx]
                time_interval = torch.Tensor([time0 - time0, time1 - time0]).to(device)
                sol = odeint(ode_func, z0, time_interval, rtol=1e-4, atol=1e-4)
                z0 = sol[-1].clone()
                solves = torch.cat([solves, sol[-1:, :]], 0).to(device)
        except AssertionError:
            print(times)
            print(time0, time1, time_interval, ptnm)
            continue
    
        preds = classifier(solves, cmax_time).permute(1, 0, 2)


        if phase == "test":
            idx_not_nan = ~(torch.isnan(labels).to(device) | (labels == -1))
            preds = preds[idx_not_nan]
            labels = labels[idx_not_nan]


            times = times[idx_not_nan.flatten()]
            ptnms += ptnm * len(times)
            Times = torch.cat((Times, times*24)).to(device)

            predictions = torch.cat((predictions, preds)).to(device)
            ground_truth = torch.cat((ground_truth, labels)).to(device)
        
        else:
            """
            time_idx = (times >= 21)
            preds = preds[:, time_idx, :]
            labels = labels[:, time_idx, :]
            """
            idx_not_nan = ~(torch.isnan(labels).to(device) | (labels == -1))
            preds = preds[idx_not_nan]
            labels = labels[idx_not_nan]

            predictions = torch.cat((predictions, preds))
            ground_truth = torch.cat((ground_truth, labels))

    rmse_loss = mean_squared_error(
        ground_truth.cpu().numpy(), predictions.cpu().numpy(),
        squared=False
    )
    r2 = r2_score(ground_truth.cpu().numpy(), predictions.cpu().numpy())

    if phase == "test":
        return {"PTNM": ptnms,
                "TIME": Times.cpu(),  
                "labels": ground_truth.cpu().tolist(), 
                "preds": predictions.cpu().tolist(),
                "loss": rmse_loss}
    else:
        return {"labels": ground_truth.cpu().tolist(), 
                "preds": predictions.cpu().tolist(),
                "loss": rmse_loss,
                "r2": r2}


def compute_loss_on_interp(encoder, ode_func, classifier, dataloader, dataloader_o, n_batches, device, phase):
    ptnms = []
    Times = torch.Tensor([]).to(device=device)
    predictions = torch.Tensor([]).to(device=device)
    ground_truth = torch.Tensor([]).to(device=device)
    latent_dim = 6

    for itr in range(n_batches):
        ptnm, times, features, labels, cmax_time = dataloader.__next__()
        ptnm_o, times_o, features_o, labels_o, cmax_time_o = dataloader_o.__next__()
        assert ptnm == ptnm_o

        dosing = torch.zeros([features.size(0), features.size(1), latent_dim]).to(device)
        dosing[:, :, 0] = features[:, :, -2]
        dosing = dosing.permute(1, 0, 2)

        encoder_out = encoder(features_o)
        qz0_mean, qz0_var = encoder_out[:, :latent_dim], encoder_out[:, latent_dim:]
        z0 = sample_standard_gaussian(qz0_mean, qz0_var)

        solves = z0.unsqueeze(0).clone()
        try:
            for idx, (time0, time1) in enumerate(zip(times[:-1], times[1:])):
                z0 += dosing[idx]
                time_interval = torch.Tensor([time0 - time0, time1 - time0]).to(device)
                sol = odeint(ode_func, z0, time_interval, rtol=1e-4, atol=1e-4)
                z0 = sol[-1].clone()
                solves = torch.cat([solves, sol[-1:, :]], 0).to(device)
        except AssertionError:
            print(times)
            print(time0, time1, time_interval, ptnm)
            continue
    
        preds = classifier(solves, cmax_time).permute(1, 0, 2)

        idx_not_nan = ~(torch.isnan(labels).to(device) | (labels == -1))
        print(idx_not_nan)
        preds = preds[idx_not_nan]
        labels = labels[idx_not_nan]

        times = times[idx_not_nan.flatten()]
        ptnms += ptnm * len(times)
        Times = torch.cat((Times, times*24)).to(device)

        predictions = torch.cat((predictions, preds)).to(device)
        ground_truth = torch.cat((ground_truth, labels)).to(device)
        
    rmse_loss = mean_squared_error(
        ground_truth.cpu().numpy(), predictions.cpu().numpy(),
        squared=False
    )
    r2 = r2_score(ground_truth.cpu().numpy(), predictions.cpu().numpy())

    return {"PTNM": ptnms,
            "TIME": Times,  
            "labels": ground_truth.cpu().tolist(), 
            "preds": predictions.cpu().tolist(),
            "loss": rmse_loss}


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# BRINGING IN THE ENTIRE MODEL.PY FILE

In [ ]:
import torch
import torch.nn as nn
# from args import args
# from torch.nn.modules.rnn import GRU, LSTM, RNN
# import utils


class ODEFunc(nn.Module):

    def __init__(self, input_dim, hidden_dim, device=torch.device("cuda")):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.SELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SELU(),
            nn.Linear(hidden_dim, input_dim)
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.001)
                nn.init.constant_(m.bias, val=0.5)

    def forward(self, t, x):
        # print(x)
        return self.net(x)


class Encoder(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim, device=torch.device("cuda")):
        super(Encoder, self).__init__()

        self.output_dim = output_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.device = device

        self.hiddens_to_output = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.output_dim),
        )
        init_network_weights(self.hiddens_to_output, std=0.001)

        # self.rnn = nn.RNN(self.input_dim, self.hidden_dim, nonlinearity="relu").to(device)
        self.rnn = nn.GRU(self.input_dim, self.hidden_dim).to(device)

    def forward(self, data):
        data = data.permute(1, 0, 2)
        data = reverse(data)
        output_rnn, _ = self.rnn(data)
        #print(output_rnn)
        outputs = self.hiddens_to_output(output_rnn[-1])
        #print(outputs)
        
        return outputs


class Classifier(nn.Module):

    def __init__(self, latent_dim, output_dim, device=torch.device("cuda")):
        super(Classifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim + 20, 32),
            nn.SELU(),
            nn.Linear(32, output_dim),
        )
        
        init_network_weights(self.net, std=0.001)

    def forward(self, z, cmax_time):
        cmax_time = cmax_time.repeat(z.size(0), 1, 1)
        z = torch.cat([z, cmax_time], 2)
        return self.net(z)


# BRINGING IN THE ENTIRE EVALUATION.PY FILE

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr


def score(df, label_col, pred_col, score_fn):
    y_true = df[label_col].values
    y_pred = df[pred_col].values
    if score_fn is mean_squared_error:
        return score_fn(y_true, y_pred, squared=False)
    elif score_fn is pearsonr:
        return score_fn(y_true, y_pred)[0]
    else:
        return score_fn(y_true, y_pred)
    

def merge_predictions(files, method="mean"):
    cols = ['PTNM','TIME','preds']
    left = pd.read_csv(files[0])
    for f in files[1:]:
        right = pd.read_csv(f)
        left = left.merge(right[cols], on=["PTNM", "TIME"], how="left")
    preds = [col for col in left.columns.values if col.startswith("preds")]
    left["pred_agg"] = left[preds].agg(method, axis=1)
    left = left[left.TIME >= 168]
    print(left.shape)
    return left

def write_score(pred_df, fold, model_name, rmse_dict, score_fn):
    res = score(pred_df, "labels", "pred_agg", score_fn)
    if not rmse_dict.get("fold", []) or rmse_dict["fold"][-1] != fold:
        rmse_dict["fold"] = rmse_dict.get("fold", []) + [fold]
    rmse_dict[model_name] = rmse_dict.get(model_name, []) + [round(res, 5)]


def main(score_type, score_fn, folds, models):
    records = {}
    for fold in folds:
        in_file = ["fold_{}/fold_{}_model_{}.csv".format(fold, fold, m) for m in models]
        predictions = merge_predictions(in_file)
        predictions.to_csv("predictions.csv", index=False)
        write_score(predictions, fold, "ensemble (mean)", records, score_fn)

        for f in in_file:
            predictions = merge_predictions([f])
            write_score(predictions, fold, f.split("/")[1], records, score_fn)
    
    df = pd.DataFrame(records)
    print(df)
    summary_df = df.drop(columns="fold").agg(["min", "max", "mean", "median"])
    print(summary_df)
    df.to_csv(f"{score_type}.txt", sep="\t", index=False)
    summary_df.to_csv(f"{score_type}_summary.txt", sep="\t", index=False)


  # main("rmse", mean_squared_error, args)
  # main("r2", r2_score, args)
  # main("correlation", pearsonr, args)


# BRINGING IN THE ENTIRE DATA_PARSE.PY FILE

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

class TDM1(Dataset):

    def __init__(self, data_to_load, label_col, feature_cols, device, phase="train"):
        self.data = pd.read_csv(data_to_load)

        self.label_col = label_col
        self.features = feature_cols
        self.device = device
        self.phase = phase
        self.data["TIME"] = self.data["TIME"] / 24

    def __len__(self):
        return self.data.PTNM.unique().shape[0] 

    def __getitem__(self, index):
        ptnm = self.data.PTNM.unique()[index]
        cur_data = self.data[self.data["PTNM"] == ptnm]
        times, features, labels, cmax_time = self.process(cur_data)
        return ptnm, times, features, labels, cmax_time

    def process(self, data):
        data = data.reset_index(drop=True)
        """
        if self.phase == "train":
            random_time = np.random.randint(low=21, high=int(data["TIME"].max()) + 1)
            data = data[data.TIME <= random_time]
        else:
            pass
        """
        if (data.DSFQ == 1).all():
            cmax_time = data.loc[data.TIME < 7, ["TIME", "PK_timeCourse"]].values.flatten()
            cmax = data.loc[data.TIME < 7, "PK_timeCourse"].max()
        else:
            cmax_time = data.loc[data.TIME < 21, ["TIME", "PK_timeCourse"]].values.flatten()
            cmax = data.loc[data.TIME < 21, "PK_timeCourse"].max()

        cmax_time_full = np.zeros((20, ))
        if len(cmax_time) <= 20:
            cmax_time_full[:len(cmax_time)] = cmax_time
        else:
            cmax_time_full[:] = cmax_time[:20]
        
        data["PK_round1"] = data["PK_round1"] / cmax

        features = data[self.features].values
        labels = data[self.label_col].values
        times = data["TIME"].values

        times = torch.from_numpy(times)
        features = torch.from_numpy(features)
        labels = torch.from_numpy(labels).unsqueeze_(-1)
        cmax_time_full = torch.from_numpy(cmax_time_full)
        return times, features, labels, cmax_time_full


def tdm1_collate_fn(batch, device=device):
    D = batch[0][2].shape[1]
    N = 1

    combined_tt, inverse_indices = torch.unique(torch.cat([ex[1] for ex in batch]),
        sorted=True, return_inverse=True)
    combined_tt = combined_tt.to(device)
    # print(combined_tt, inverse_indices)

    offset = 0
    combined_features = torch.zeros([len(batch), len(combined_tt), D]).to(device)
    combined_label = torch.zeros([len(batch), len(combined_tt), N]).to(device)
    combined_label[:] = np.nan
    combined_cmax_time = torch.zeros([len(batch), 20]).to(device)
    # print(combined_label.shape)

    ptnms = []
    for b, (ptnm, tt, features, label, cmax_time) in enumerate(batch):
        ptnms.append(ptnm)
        tt = tt.to(device)
        features = features.to(device)
        label = label.to(device)
        cmax_time = cmax_time.to(device)

        indices = inverse_indices[offset:offset + len(tt)]
        offset += len(tt)

        combined_features[b, indices] = features.float()
        combined_label[b, indices] = label.float()
        combined_cmax_time[b, :] = cmax_time.float()
    combined_tt = combined_tt.float()

    return ptnms, combined_tt, combined_features, combined_label, combined_cmax_time


def parse_tdm1(device, phase="train"):
    train_data_path = "train.csv"
    val_data_path = "validate.csv"
    test_data_path = "test.csv"

    feature_cols = ['TFDS','TIME','CYCL','AMT',"PK_round1"]
    """
    covariates = ['SEX','AGE','WT','RACR','RACE','BSA',
                  'BMI','ALBU','TPRO','WBC','CRCL','CRET',
                  'SGOT','SGPT','TBIL','TMBD','ALKP', 'HER', 
                  'ECOG','KEOALL','ASIAN']
    feature_cols += covariates
    """
    label_col = "PK_timeCourse"
    train = TDM1(train_data_path, label_col, feature_cols, device, phase="train")
    validate = TDM1(val_data_path, label_col, feature_cols, device, phase="validate")
    test = TDM1(test_data_path, label_col, feature_cols, device, phase="test")

    ptnm, times, features, labels, cmax_time = train[0]
    input_dim = features.size(-1)
    # n_labels = 1

    if phase == "train":
        train_dataloader = DataLoader(train, batch_size=1, shuffle=True, 
            collate_fn=lambda batch: tdm1_collate_fn(batch, device))
        val_dataloader = DataLoader(validate, batch_size=1, shuffle=False,
            collate_fn=lambda batch: tdm1_collate_fn(batch, device))

        dataset_objs = {
            "train_dataloader": inf_generator(train_dataloader),
            "val_dataloader": inf_generator(val_dataloader),
            "n_train_batches": len(train_dataloader),
            "n_val_batches": len(val_dataloader),
            "input_dim": input_dim
        }

    else:
        test_dataloader = DataLoader(test, batch_size=1, shuffle=False,
            collate_fn=lambda batch: tdm1_collate_fn(batch, device))

        dataset_objs = {
            "test_dataloader": inf_generator(test_dataloader),
            "n_test_batches": len(test_dataloader),
            "input_dim": input_dim
        }

    return dataset_objs

    




# if __name__ == "__main__":
#     print("run")
#     data = parse_tdm1("cpu")
#     for ptnms, times, features, labels, cmax_time in data["train_dataloader"]:
#         print(ptnms)
#         print(times)
#         print(features)
#         print(labels)
#         print(cmax_time)
#         break


# BRINGING IN THE ENTIRE RUNTRAIN.PY FILE

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from random import SystemRandom
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint_adjoint as odeint
from scipy.stats import pearsonr

# import utils
# from model import *
# from data_parse import parse_tdm1
# from args import args


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
file_name = os.path.basename("__file__")[:-3]


input_cmd = sys.argv
input_cmd = " ".join(input_cmd)

########################################################################
## Main runnings
def runtraining(random_seed, model, fold, lr=0.00005, l2=0.1, epochs=30, tol=1e-4, continue_train=False):
  save = f"Fold{fold}"
  makedirs(save)
  torch.manual_seed(random_seed + model + fold)
  np.random.seed(random_seed + model + fold)

  ckpt_path = os.path.join(save, f"fold_{fold}_model_{model}.ckpt")
  ########################################################################
  tdm1_obj = parse_tdm1(device, phase="train")
  input_dim = tdm1_obj["input_dim"]
  hidden_dim = 128
  latent_dim = 6

  encoder = Encoder(input_dim=input_dim, output_dim=2 * latent_dim, hidden_dim=hidden_dim).to(device)
  ode_func = ODEFunc(input_dim=latent_dim, hidden_dim=16).to(device)
  classifier = Classifier(latent_dim=latent_dim, output_dim=1).to(device)

  # torch.save({'encoder': encoder.state_dict(), 'ode': ode_func.state_dict(), 'classifier': classifier.state_dict()}, ckpt_path)

  if continue_train:
      load_model(ckpt_path, encoder, ode_func, classifier, device)

  ########################################################################
  ## Train
  log_path = "logs/" + f"fold_{fold}_model_{model}.log"
  makedirs("logs/")
  logger = get_logger(logpath=log_path, filepath=os.path.abspath("__file__"))
  logger.info(input_cmd)

  batches_per_epoch = tdm1_obj["n_train_batches"]
  criterion = nn.MSELoss().to(device=device)
  params = (list(encoder.parameters()) + 
            list(ode_func.parameters()) + 
            list(classifier.parameters()))
  optimizer = optim.Adam(params, lr=lr, weight_decay=l2)
  best_rmse = 0x7fffffff
  best_epochs = 0

  for epoch in range(1, epochs):

      for _ in tqdm(range(batches_per_epoch), ascii=True):
          optimizer.zero_grad()

          ptnms, times, features, labels, cmax_time = tdm1_obj["train_dataloader"].__next__()
          dosing = torch.zeros([features.size(0), features.size(1), latent_dim]).to(device)
          dosing[:, :, 0] = features[:, :, -2]
          dosing = dosing.permute(1, 0, 2)

          encoder_out = encoder(features)
          qz0_mean, qz0_var = encoder_out[:, :latent_dim], encoder_out[:, latent_dim:]
          z0 = sample_standard_gaussian(qz0_mean, qz0_var).to(device)
          
          solves = z0.unsqueeze(0).clone()
          try:
              for idx, (time0, time1) in enumerate(zip(times[:-1], times[1:])):
                  z0 += dosing[idx]
                  # print(z0.device)
                  time_interval = torch.Tensor([time0 - time0, time1 - time0]).to(device)
                  sol = odeint(ode_func, z0, time_interval, rtol=tol, atol=tol).to(device)
                  z0 = sol[-1].clone()
                  solves = torch.cat([solves, sol[-1:, :]], 0).to(device)
          except AssertionError:
              print(times)
              print(time0, time1, time_interval, ptnms)
              continue
      
          preds = classifier(solves, cmax_time)

          loss = compute_loss_on_train(criterion, labels, preds)
          try: 
              loss.backward()
          except RuntimeError:
              print(ptnms)
              print(times)
              continue
          optimizer.step()
      
      idx_not_nan = ~(torch.isnan(labels) | (labels == -1))
      preds = preds.permute(1, 0, 2)[idx_not_nan]
      labels = labels[idx_not_nan]
      print(preds)
      print(labels)

      with torch.no_grad():
          
          train_res = compute_loss_on_test(encoder, ode_func, classifier,
              tdm1_obj["train_dataloader"], tdm1_obj["n_train_batches"], 
              device, phase="train")

          validation_res = compute_loss_on_test(encoder, ode_func, classifier,
              tdm1_obj["val_dataloader"], tdm1_obj["n_val_batches"], 
              device, phase="validate")
          
          train_loss = train_res["loss"] 
          validation_loss = validation_res["loss"]
          if validation_loss < best_rmse:
              torch.save({'encoder': encoder.state_dict(),
                          'ode': ode_func.state_dict(),
                          'classifier': classifier.state_dict()}, ckpt_path)
              best_rmse = validation_loss
              best_epochs = epoch

          message = """
          Epoch {:04d} | Training loss {:.6f} | Training R2 {:.6f} | Validation loss {:.6f} | Validation R2 {:.6f}
          Best loss {:.6f} | Best epoch {:04d}
          """.format(epoch, train_loss, train_res["r2"], validation_loss, validation_res["r2"], best_rmse, best_epochs)
          logger.info(message)
            

#BRINGING IN THE ENTIRE RUNPREDICT.PY FILE

In [ ]:
import os
import numpy as np
import pandas as pd
import torch



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###############################################################
## Main runnings
def runpredict(random_seed, model, fold, lr=0.00005, l2=0.1, epochs=30, tol=1e-4, continue_train=True):
  save = f"Fold{fold}"
  makedirs(save)
  ckpt_path = os.path.join(save, f"fold_{fold}_model_{model}.ckpt")
  eval_path = os.path.join(save, f"fold_{fold}_model_{model}.csv")
  res_path = "rmse.csv"

  ########################################################################
  tdm1_obj = parse_tdm1(device, phase="test")
  input_dim = tdm1_obj["input_dim"]
  hidden_dim = 128 
  latent_dim = 6

  encoder = Encoder(input_dim=input_dim, output_dim=2 * latent_dim, hidden_dim=hidden_dim)
  ode_func = ODEFunc(input_dim=latent_dim, hidden_dim=16)
  classifier = Classifier(latent_dim=latent_dim, output_dim=1)

  load_model(ckpt_path, encoder, ode_func, classifier, device)

  ########################################################################
  ## Predict & Evaluate
  with torch.no_grad():
      test_res = compute_loss_on_test(encoder, ode_func, classifier,
          tdm1_obj["test_dataloader"], tdm1_obj["n_test_batches"], 
          device, phase="test")
      

  eval_results = pd.DataFrame(test_res).drop(columns="loss")
  eval_results.to_csv(eval_path, index=False)

  """
  with torch.no_grad():
      test_res = utils.compute_loss_on_interp(encoder, ode_func, classifier, args,
          tdm1_obj["interp_dataloader"], tdm1_obj["test_dataloader"], tdm1_obj["n_interp_batches"], 
          device, phase="test")

  eval_results = pd.DataFrame(test_res).drop(columns="loss")
  eval_results.to_csv(eval_path + ".interp", index=False)

  with torch.no_grad():
      test_res = utils.compute_loss_on_interp(encoder, ode_func, classifier, args,
          tdm1_obj["nodosing_dataloader"], tdm1_obj["test_dataloader"], tdm1_obj["n_interp_batches"], 
          device, phase="test")

  eval_results = pd.DataFrame(test_res).drop(columns="loss")
  eval_results.to_csv(eval_path + ".nodosing", index=False)
  """

RUNNING THE NEURAL-ODE MODEL

In [ ]:
fold, model = 1,1
train, validate, test = datasplitter(data_complete, fold, model)
ptnms = ['5010','5011','5012','5013','5014','5015','5016','5209','5210','5211','5212','5409','5410','5411','5412','5413','5414','5415','5416','5602','5603','5604','5605','5606','5607','5608','5609','5610']
test_interp = pd.DataFrame()
for ptnm in ptnms:
    print(ptnm)
    test_interp = pd.concat([test_interp, generate_interp(int(ptnm))], ignore_index=True)
test_interp.to_csv("test_interp.csv", index=False)
test = pd.read_csv("test.csv")
test.loc[test.CYCL > 5, "AMT"] = 0
test.to_csv("test_nodosing.csv", index=False)
runtraining(1000, model, fold, 0.00005, 0.1, 30, 1e-4, False)
runpredict(1000, model, fold, 0.00005, 0.1, 30, 1e-4, True)

print(fold, model)

<ipython-input-13-a1547cc7bd9e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PTNM"] = df["PTNM"] + str(0.1)
<ipython-input-13-a1547cc7bd9e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PTNM"] = df["PTNM"] + str(0.2)
<ipython-input-13-a1547cc7bd9e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

5010
5011
5012
5013
5014
5015
5016
5209
5210
5211
5212
5409
5410
5411
5412
5413
5414
5415
5416
5602
5603
5604
5605
5606
5607
5608
5609
5610


INFO:root:/content/__file__
/content/__file__
INFO:root:/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-395d1998-8e4e-4dc9-a7fa-cdd8edebe74a.json
/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-395d1998-8e4e-4dc9-a7fa-cdd8edebe74a.json
100%|##########| 672/672 [04:43<00:00,  2.37it/s]


tensor([ 0.2391,  5.1717,  8.8703,  9.2620, 12.9289, 13.1532, 13.3494, 17.0739,
        17.2425], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([ 5.8156,  0.8816, 16.1310,  0.8871, 19.8420,  3.7824,  0.8870, 20.1590,
         5.8156], device='cuda:0')


INFO:root:
          Epoch 0001 | Training loss 26.341450 | Training R2 -0.768458 | Validation loss 28.023655 | Validation R2 -1.255971
          Best loss 28.023655 | Best epoch 0001
          

          Epoch 0001 | Training loss 26.341450 | Training R2 -0.768458 | Validation loss 28.023655 | Validation R2 -1.255971
          Best loss 28.023655 | Best epoch 0001
          
100%|##########| 672/672 [04:40<00:00,  2.39it/s]


...

tensor([ 8.3549, 41.2138, 34.2881, 29.6694, 25.6575, 15.7572, 47.6375, 30.6324,
        18.5476, 50.3466, 31.9987, 18.9066, 51.0366, 43.9544, 18.6381, 43.7398,
        18.6526, 43.3545, 18.4825, 18.2349, 49.8410, 18.2371, 42.3167, 18.1987],
       device='cuda:0', grad_fn=<IndexBackward0>)
tensor([12.5700, 31.8440, 25.2520, 20.7650, 17.3880, 10.5840, 40.6760, 22.8730,
        13.9690, 43.5540, 24.6480, 15.0630, 44.0460, 35.6960, 15.2700, 35.8460,
        15.3380, 35.8950, 15.3590, 15.2690, 43.9280, 15.2390, 35.5910, 15.2300],
       device='cuda:0')


INFO:root:
          Epoch 0028 | Training loss 6.356060 | Training R2 0.897035 | Validation loss 5.972003 | Validation R2 0.897547
          Best loss 5.972003 | Best epoch 0028
          

          Epoch 0028 | Training loss 6.356060 | Training R2 0.897035 | Validation loss 5.972003 | Validation R2 0.897547
          Best loss 5.972003 | Best epoch 0028
          
100%|##########| 672/672 [07:49<00:00,  1.43it/s]


tensor([ 6.9066, 82.5752, 25.0994,  7.8536,  3.5957, 18.4058,  4.5519, 59.7573,
        22.0278,  7.7739,  5.0958, 68.1590,  5.6257, 21.3132,  9.0211,  6.2920,
        67.1497, 21.4841,  9.6844,  6.7338, 66.4735,  9.6566,  6.8122, 64.8901,
        21.4822,  9.3902,  6.9110, 62.5967, 20.4064,  9.1618,  6.9159, 52.6548,
        20.6670,  9.3857,  6.9895, 60.4102, 23.1409,  9.2480], device='cuda:0',
       grad_fn=<IndexBackward0>)
tensor([ 6.1430, 77.5480, 24.0970,  7.7805,  2.9525, 17.9790,  3.0428, 55.8410,
        21.1480,  6.8284,  3.0454, 63.6220,  2.9330, 20.3360,  6.5662,  2.9284,
        63.5080, 20.3310,  6.5647,  2.9278, 63.5290,  6.5668,  2.9287, 62.7370,
        20.0850,  6.4851,  2.8923, 61.1750, 19.5850,  6.3236,  2.8202, 51.6830,
        19.5730,  6.3199,  2.8186, 59.4280, 22.3600,  6.1430], device='cuda:0')


INFO:root:
          Epoch 0029 | Training loss 6.130345 | Training R2 0.904218 | Validation loss 5.422178 | Validation R2 0.915544
          Best loss 5.422178 | Best epoch 0029
          

          Epoch 0029 | Training loss 6.130345 | Training R2 0.904218 | Validation loss 5.422178 | Validation R2 0.915544
          Best loss 5.422178 | Best epoch 0029
          


1 1


In [16]:
fold, model = 1,1
train, validate, test = datasplitter(data_complete, fold, model)
ptnms = ['5010','5011','5012','5013','5014','5015','5016','5209','5210','5211','5212','5409','5410','5411','5412','5413','5414','5415','5416','5602','5603','5604','5605','5606','5607','5608','5609','5610']
test_interp = pd.DataFrame()
for ptnm in ptnms:
    print(ptnm)
    test_interp = pd.concat([test_interp, generate_interp(int(ptnm))], ignore_index=True)
test_interp.to_csv("test_interp.csv", index=False)
test = pd.read_csv("test.csv")
test.loc[test.CYCL > 5, "AMT"] = 0
test.to_csv("test_nodosing.csv", index=False)
x_train = train[["CYCL", "AMT", "TIME", "TFDS"]]
y_train = train["PK_timeCourse"]
x_val = validate[["CYCL", "AMT", "TIME", "TFDS"]]
y_val = validate["PK_timeCourse"]
x_test = test[["CYCL", "AMT", "TIME", "TFDS"]]
y_test = test["PK_timeCourse"]
train

<ipython-input-13-a1547cc7bd9e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PTNM"] = df["PTNM"] + str(0.1)
<ipython-input-13-a1547cc7bd9e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PTNM"] = df["PTNM"] + str(0.2)
<ipython-input-13-a1547cc7bd9e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

5010
5011
5012
5013
5014
5015
5016
5209
5210
5211
5212
5409
5410
5411
5412
5413
5414
5415
5416
5602
5603
5604
5605
5606
5607
5608
5609
5610


,STUD,DSFQ,PTNM,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID,PK_round1
0,1000.0,3.0,00001,1.0,296.6,0.0,0.0,20.3820,100000001,20.382
1,1000.0,3.0,00001,1.0,0.0,24.0,24.0,73.1480,100000001,73.148
2,1000.0,3.0,00001,1.0,0.0,216.0,216.0,19.7640,100000001,19.764
3,1000.0,3.0,00001,1.0,288.0,504.0,504.0,3.2199,100000001,0.000
4,1000.0,3.0,00001,2.0,0.0,696.0,192.0,23.2100,100000001,0.000
...,...,...,...,...,...,...,...,...,...,...
7790,3000.0,3.0,001960.1,1.0,0.0,24.0,24.0,53.9210,300000196,53.921
7791,3000.0,3.0,001960.2,1.0,217.0,0.0,0.0,1.7780,300000196,1.778
7792,3000.0,3.0,001960.2,1.0,0.0,24.0,24.0,53.9210,300000196,53.921
7793,3000.0,3.0,001960.3,1.0,217.0,0.0,0.0,1.7780,300000196,1.778


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

# Assuming you have trained the model and made predictions on the validation set


params = {
    'boosting_type': 'gbdt',
    'learning_rate': 0.005,
    'num_leaves': 30,
    'num_trees': 1000,
    'objective': 'regression',
    'metric': 'rmse'
}  

model = lgb.LGBMRegressor(
    boosting_type= 'gbdt',
    learning_rate=0.005,
    num_leaves= 30,
    num_trees=1000,
    objective='regression',
    num_boost_round=10000,
    early_stopping_rounds=50, 
    verbose_eval=100)

model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],
          verbose=20,eval_metric='rmse')

# train_data = lgb.Dataset(train_x, train_y)

# # Train the model
# model = lgb.train(train_data,boosting_type= 'gbdt',
#     learning_rate=0.005,
#     num_leaves= 30,
#     num_trees=1000,
#     objective='regression',
#     metric='rmse',num_boost_round=10000, 
#     valid_sets=[train_data, (val_x, val_y)], 
#                   early_stopping_rounds=50, 
#                   verbose_eval=100)

y_pred = model.predict(x_test)

# Evaluate the model using RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse:.4f}")

y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
print("R2 Score: {:.2f}".format(r2))

pearson_corr, _ = pearsonr(y_test, y_pred)
print(f"Pearson correlation coefficient: {pearson_corr:.4f}")

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[20]	training's l2: 335.381	training's rmse: 18.3134	valid_0's l2: 341.92	valid_0's rmse: 18.4911
[40]	training's l2: 288.729	training's rmse: 16.992	valid_0's l2: 313.52	valid_0's rmse: 17.7065
[60]	training's l2: 250.552	training's rmse: 15.8288	valid_0's l2: 293.22	valid_0's rmse: 17.1237
[80]	training's l2: 219.233	training's rmse: 14.8065	valid_0's l2: 279.207	valid_0's rmse: 16.7095
[100]	training's l2: 193.427	training's rmse: 13.9078	valid_0's l2: 270.042	valid_0's rmse: 16.433
[120]	training's l2: 172.24	training's rmse: 13.124	valid_0's l2: 264.441	valid_0's rmse: 16.2616
[140]	training's l2: 154.808	training's rmse: 12.4422	valid_0's l2: 262.174	valid_0's rmse: 16.1918
[160]	training's l2: 140.481	training's rmse: 11.8525	valid_0's l2: 262.269	valid_0's rmse: 16.1947
[180]	training's l2: 128.623	training's rmse: 11.3412	valid_0's l2: 264.735	valid_0's rmse: 16.2707
Early stopping, best iteration is:
[149]	training

##LSTM Model

In [17]:
lstm_train_x = train[["CYCL", "AMT", "TIME", "TFDS", "PK_round1"]]
lstm_train_y = train[["PK_timeCourse"]]
twentyelem = torch.flatten(torch.tensor(lstm_train_x[["TIME", "PK_round1"]].head(20).values, dtype=torch.float32))

In [18]:
class LSTM(nn.Module):
      def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        # LSTM layers 1 and 2
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        
        # Decoder
        self.decodernet = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU(), 
            nn.Linear(128, output_dim))
        init_network_weights(self.decodernet)
      
      def forward(self, x):
        output, _ = self.lstm1(x)
        output = torch.tanh(output)
        output, _ = self.lstm2(output)
        output = torch.tanh(output)

        #concatenate with PK_cycle1
        print(output)
        #output = torch.cat((output, twentyelem))

        output = self.decodernet(output)

        return output


# Define hyperparameters
input_dim = 5
hidden_dim = 128
output_dim = 1
batch_size = 1
learning_rate = 0.00005
weight_decay = 0.1
num_epochs = 30
seq_len = 5

#initializing everything
model = LSTM(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.1)
criterion = nn.MSELoss()


#train loop
for epoch in range(num_epochs):
  optimizer.zero_grad()
  x = torch.tensor(lstm_train_x.values, dtype=torch.float32)
  y = torch.tensor(lstm_train_y.values, dtype=torch.float32)

  output = model(x)
  loss = criterion(output, y)
  loss.backward()
  optimizer.step()
        
  print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch+1, len(train), loss.item()**(0.5)))

tensor([[ 0.0355,  0.0066, -0.1040,  ..., -0.0288,  0.0118, -0.0357],
        [ 0.0595, -0.0376, -0.0706,  ...,  0.0110,  0.1287,  0.0078],
        [ 0.0361, -0.0480, -0.1347,  ..., -0.0966,  0.0580,  0.0037],
        ...,
        [-0.0015, -0.0148, -0.1271,  ..., -0.0615,  0.1382,  0.0100],
        [-0.0243,  0.0322, -0.1807,  ..., -0.1305,  0.1057,  0.0029],
        [-0.0006, -0.0152, -0.1270,  ..., -0.0610,  0.1386,  0.0098]],
       grad_fn=<TanhBackward0>)
Epoch [1/30], Batch [1/7795], Loss: 27.7238
tensor([[ 3.5834e-02,  6.6643e-03, -1.0430e-01,  ..., -2.8557e-02,
          1.1468e-02, -3.5510e-02],
        [ 5.9606e-02, -3.7527e-02, -7.0859e-02,  ...,  1.1037e-02,
          1.2809e-01,  8.2480e-03],
        [ 3.5928e-02, -4.8620e-02, -1.3540e-01,  ..., -9.5884e-02,
          5.7191e-02,  4.8181e-03],
        ...,
        [-9.9317e-04, -1.5084e-02, -1.2771e-01,  ..., -6.0964e-02,
          1.3744e-01,  1.0547e-02],
        [-2.3692e-02,  3.2045e-02, -1.8126e-01,  ..., -1.2988e-01

In [ ]:
# for fold in np.arange(1,6):
#   for model in np.arange(1,6):
#      train, validate, test = datasplitter(data_complete, fold, model)
#      runtraining(1000, model, fold, 0.00005, 0.1, 30, 1e-4, True)
#      runpredict(1000, model, fold, 0.00005, 0.1, 30, 1e-4, True)

#      print(fold, model)

In [ ]:
complete_data

NameError: ignored

In [ ]:
for fold in np.arange(1,6):
  for model in np.arange(1,6):
     train, validate, test = datasplitter(data_complete, fold, model)
     

     print(fold, model)